# Retrieval Augmented Generation (RAG) using New Hampshire Case Law
*With IBM Granite Models*

The [New Hampshire Case Law Dataset](https://huggingface.co/datasets/free-law/nh) comes from the Caselaw Access Project via Hugging Face.

## In this notebook
This notebook contains instructions for performing Retrieval Augumented Generation (RAG). RAG is an architectural pattern that can be used to augment the performance of language models by recalling factual information from a knowledge base, and adding that information to the model query. The most common approach in RAG is to create dense vector representations of the knowledge base in order to retrieve text chunks that are semantically similar to a given user query.

RAG use cases include:
- Customer service: Answering questions about a product or service using facts from the product documentation.
- Domain knowledge: Exploring a specialized domain (e.g., finance) using facts from papers or articles in the knowledge base.
- News chat: Chatting about current events by calling up relevant recent news articles.

In its simplest form, RAG requires 3 steps:

- Initial setup:
  - Index knowledge-base passages for efficient retrieval. In this recipe, we take embeddings of the passages using WatsonX, and store them in a vector database.
- Upon each user query:
  - Retrieve relevant passages from the database. In this recipe, we using an embedding of the query to retrieve semantically similar passages.
  - Generate a response by feeding retrieved passage into a large language model, along with the user query.

## Prerequisites

To get started, you'll need:
* A [Replicate account](https://replicate.com/) and API token.

## Setting up the environment

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for notebooks. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [ ]:
!pip install git+https://github.com/ibm-granite-community/granite-kitchen \
    datasets \
    langchain-huggingface \
    langchain-milvus

## Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text.

To use a model from a provider other than Huggingface, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/utils/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus, substitute this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/utils/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [ ]:
from langchain_milvus import Milvus
import uuid

db_file = f"/tmp/milvus_{str(uuid.uuid4())[:8]}.db"
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(embedding_function=embeddings_model, connection_args={"uri": db_file}, auto_id=True)

### Choose your LLM
The LLM will be used for answering the question, given the retrieved text.

Follow the instructions in [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/cee1513c77429d7ddbf0e5a49b29b7bc9ca0d996/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb), selecting a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org.

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [ ]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import set_env_var

set_env_var("REPLICATE_API_TOKEN")

model = Replicate(
    model="ibm-granite/granite-8b-code-instruct-128k",
)

## Acquiring the Data

We will use a New Hampshire case law dataset to help the model answer questions about NH laws.

### Download the documents

Download the [New Hampshire CAP Caselaw](https://huggingface.co/datasets/free-law/nh) dataset from HuggingFace using the datasets library.

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader

# Load the documents from the dataset
loader = HuggingFaceDatasetLoader("free-law/nh", page_content_column="text")
documents = loader.load()
print("Document Count: " + str(len(documents)))

### Add metadata to the documents

Add the `source` field, which is used below, to the metadata.

In [ ]:
for doc in documents:
    doc.metadata['source'] = doc.metadata['id']

### Inspect the documents

In [ ]:
for doc in documents[:1]:
    print(doc.metadata, "\n")
    print(doc.page_content, "\n")

## Building the Document Database

We'll use the caselaw document database to retrieve the full text of the cases by case id.

### Create the database file and document table

In [ ]:
# put the json objects in a sqlite database, keyed by id
import sqlite3, os, json

# remove database file if exists
if os.path.isfile('data.db'):
    os.remove('data.db')

conn = sqlite3.connect('data.db')
c = conn.cursor()

# create the table if it doesn't exist. include id, text, and size
c.execute('''CREATE TABLE IF NOT EXISTS data
             (id INTEGER PRIMARY KEY UNIQUE,
              metadata TEXT,
              text TEXT,
              char_count INTEGER)''')


### Insert the documents into the table

In [ ]:
for doc in documents:
    id = doc.metadata["id"]
    c.execute("INSERT INTO data (id, metadata, text, char_count) VALUES (?,?,?,?)", (id, json.dumps(doc.metadata), doc.page_content, doc.metadata["char_count"]))
    conn.commit()

### Count the documents

In [ ]:
c.execute("SELECT count(*) FROM data")
doc_count = c.fetchone()[0]
print(f"Document count: {doc_count}")

## Building the Vector Database

In this example, we take the caselaw text, split it into chunks, derive embedding vectors using the embedding model, and load it into the vector database for querying.

### Split the document into chunks

Split the document into text segments that can fit into the model's context window.

In [ ]:
from langchain.text_splitter import TokenTextSplitter

# Split the documents into chunks
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10)
chunks = text_splitter.split_documents(documents)
print("Chunk Count: " + str(len(chunks)))

### Inspect the chunks

In [ ]:
import json
for i in range(1):
    print(chunks[i].page_content)
    print(json.dumps(chunks[i].metadata, indent=4))

### Populate the vector database

NOTE: Population of the vector database may take a few minutes depending on your embedding model and service.

In [ ]:
ids = vector_db.add_documents(chunks)
print("Document IDs: " + str(ids[:3]))

## Querying the Databases

### Create query text

Here we use the text of a NH law to query into the vector database for relevant cases.

In [ ]:
# https://schoolsafetyresources.nh.gov/wp-content/uploads/2019/02/New-Hampshire-School-Discipline-Laws-and-Regulations.pdf
# https://www.gencourt.state.nh.us/rsa/html/XV/193/193-13.htm
law_id = "193:13"
law_name = "Suspension and Expulsion of Pupils."
law_text = """
School Attendance
Section 193:13
    193:13 Suspension and Expulsion of Pupils. –
I. (a) A superintendent or chartered public school director, or a representative designated in writing by the superintendent or chartered public school director, may suspend pupils from school for a period not to exceed 10 consecutive school days for:
(1) Behavior that is detrimental to the health, safety, or welfare of pupils or school personnel; or
(2) Repeated and willful disregard of the reasonable rules of the school that is not remediated through imposition of the district's graduated sanctions under paragraph X.
(b) The school board or chartered public school board of trustees, or a representative designated in writing may, following a hearing, extend the suspension of a pupil up to 10 additional consecutive school days for an act that constitutes an act of theft, destruction, or violence as defined in RSA 193-D; bullying pursuant to school district policy when the pupil has not responded to targeted interventions and poses an ongoing threat to the safety or welfare of another student; or possession of a firearm, BB gun, or paintball gun. The school board's or board of trustee's designee may be the superintendent or any other individual, but may not be the individual who suspended the pupil for the first 10 days under subparagraph (a). Any suspension shall be valid throughout the school districts of the state, subject to modification by the superintendent of the school district or chartered public school in which the pupil seeks to enroll.
(c) Any suspension in excess of 10 school days imposed under subparagraph (b) by any person other than the school board or board of trustees is appealable to the school board or board of trustees, provided that the superintendent, school board, or board of trustees received such appeal in writing within 10 days after the issuance of the decision being appealed. The school board or board of trustees shall hold a hearing on the appeal, but shall have discretion to hear evidence or to rely upon the record of a hearing conducted under subparagraph (b). The suspension under subparagraph (b) shall be enforced while that appeal is pending, unless the school board or board of trustees stays the suspension while the appeal is pending.
II. Any pupil may be expelled from school by the local school board or board of trustees for an act that poses an ongoing threat to the safety of students or school personnel and that constitutes:
(a) A repeated act under subparagraph I(b);
(b) Any act of physical or sexual assault that would be a felony if committed by an adult;
(c) Any act of violence pursuant to RSA 651:5, XIII; or
(d) Criminal threatening pursuant to RSA 631:4, II(a).
III. A pupil who has been expelled shall not attend school until reinstated by the local board or chartered public school board of trustees.
III-a. Before expelling a pupil under this section the local school board or chartered public school board of trustees shall consider each of the following factors:
(a) The pupil's age.
(b) The pupil's disciplinary history.
(c) Whether the pupil is a student with a disability.
(d) The seriousness of the violation or behavior committed by the pupil.
(e) Whether the school district or chartered public school has implemented positive behavioral interventions under paragraph V.
(f) Whether a lesser intervention would properly address the violation or behavior committed by the pupil.
III-b. Any expulsion shall be subject to review by the pupil's school board of attendance or the board of trustees of the chartered public school's board that issued the expulsion if requested prior to the start of each school year and further, any parent or guardian has the right to appeal any such expulsion by the local board or board of trustees to the state board of education at any time while the expulsion remains in effect. All appeals of final action by the state board of education shall be in accordance with RSA 541.
III-c. Any expulsion shall be valid throughout the school districts of the state. However, upon application by the pupil, any school district or chartered public school may choose to admit an expelled pupil at the school district or chartered public school's sole discretion. The decision by a chartered public school or superintendent to accept a pupil under this paragraph shall not be binding upon any other school district or chartered public school until the pupil is reinstated by the pupil's local school board or chartered public school board of trustees.
IV. Any pupil who brings or possesses a firearm as defined in section 921 of Title 18 of the United States Code in a safe school zone as defined in RSA 193-D:1 without written authorization from the superintendent or designee shall be expelled from school by the local school board for a period of not less than 12 months. Nothing in this section shall be construed to prevent the local school district or chartered public school that expelled the student from providing educational services to such student in an alternative setting.
V. School districts and chartered public schools shall make educational assignments available to the suspended pupil during periods of suspension. Except as provided in paragraphs II and IV, a school district or chartered public school shall provide alternative educational services to a suspended pupil whenever the pupil is suspended in excess of 20 cumulative days within any school year. The alternative educational services shall be designed to enable a pupil to advance from grade to grade. Any time a pupil is suspended more than 10 school days in any school year, upon the pupil's return to school the school district shall develop an intervention plan designed to proactively address the pupil's problematic behaviors. No pupil shall be penalized academically solely by virtue of missing class due to suspension.
VI. A pupil expelled from school in another state under the provisions of the Gun-Free Schools Act of 1994 shall not be eligible to enroll in a school district in New Hampshire for the period of such expulsion. If the out-of-state expulsion is for an indefinite period of time, such pupil or the pupil's parent or guardian shall have the right to petition the pupil's local school board for enrollment upon establishing residency. If the pupil is denied enrollment, the pupil's expulsion shall be subject to review pursuant to paragraph III-b.
VII. The local school board or chartered public school shall adopt a policy which allows the superintendent or charter public school director to modify the expulsion and enrollment requirements under paragraphs IV and VI on a case by case basis.
VIII. For purposes of paragraphs I, II, III, and IV school board may be either the school board or a subcommittee of the board duly authorized by the school board.
IX. Nothing in this section shall prevent the superintendent of the pupil's local school district or chartered public school director from reinstating a suspended or expelled pupil.
X. The provisions of this section shall be construed in a manner consistent with RSA 186-C.
XI. School boards and chartered public schools shall establish policies on school discipline that contain a system of supports and consequences designed to correct student misconduct and promote behavior within acceptable norms. Such policies shall:
(a) Include a graduated set of age appropriate responses to misconduct that may include, but are not limited to, parent conferences, counseling, peer mediation, instruction in conflict resolution and anger management, parent counseling and training, community service, rearranging class schedules, restriction from extra curricular activities, detention, in-school supports and consequences, out-of-school suspension, and expulsion.
(b) Set forth standards for short term suspensions up to 5 days, short term suspensions up to 10 days, long term suspensions up to 20 days, and expulsion. Such standards shall make reference to the nature and degree of disruption caused to the school environment, the threat to the health and safety of pupils and school personnel, and the isolated or repeated nature of incidents forming the basis of disciplinary action.
XII. Each school district and chartered public school shall make its policy on school discipline:
(a) Available to parents at the beginning of each school year;
(b) Publicly available on the district, school administrative unit, or chartered public school website and in the student handbook; and
(c) Available to parents via a manner designed to ensure parental notification if the school district, school administrative unit, or chartered public school does not maintain a website and/or student handbook."""

### Query the vector database

Query the vector database for cases related to the law. Similar documents are found by proximity of the embedded vector in vector space.

In [ ]:
k = 10  # the number of docs to retrieve
docs_with_score = vector_db.similarity_search_with_score(law_text, k=k)

# Get a unique set of docs.
docs = []
doc_ids = {}
for doc, score in docs_with_score:
    # print(doc.metadata["name_abbreviation"])
    # print(score)
    id = doc.metadata["id"]
    if id not in doc_ids:
        docs.append(doc)
        print(id)
        print(doc.metadata["name_abbreviation"])
        doc_ids[id] = 1

### Query the document database

Get the full text of the first case found by the vector search.

In [ ]:
case_id = docs[0].metadata["id"]
case_short_name = docs[0].metadata["name_abbreviation"]
print(f"Case Id: {case_id}")
print(f"Case Name: {case_short_name}")

c.execute("SELECT text FROM data where id = ?", (case_id,))
case_text = c.fetchone()[0]
print(f"Case Text Length: {len(case_text)} chars")

## Answering Questions

### Assemble the Chat Prompt

Build a chat prompt template with the law and the retrieved case.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_prompt = (
    "You are an assistant for question-answering tasks. "
    f"Below is the text of NH law '{law_id}', followed by a related NH court case. "
    f"Use only the NH court case to answer the question. "
    "If you can't answer the question with these court case, say that you don't know."
    "\n\n"
    "Here is the text of the law:\n"
    f"{law_text}"
    "\n\n"
    f"Here is the related NH court case:\n"
    f"{case_text}"
)

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

rag_chain = (
    rag_prompt
    | model
    | StrOutputParser()
)

### Ask questions of the retrieved case in relation to the law.

First summarize the case.

In [ ]:
query = f"Summarize the NH court case according to the IRAC framework (Issue, Rule, Application, Conclusion)."

response = rag_chain.invoke(input = query)
print(response)

Next, discuss how the law was applied to the case.

In [ ]:
query = "How was NH law 193:13 applied to this case?"
response = rag_chain.invoke(query)
print(response)

In [ ]:
query = "What were the facts presented in the case and how are they relevant to the law?"
response = rag_chain.invoke(query)
print(response)